# Initializing a model

Example showing how to initialize a model with another model

In [2]:
!pip install pybamm -q    # install PyBaMM if it is not installed

import pybamm
import pandas as pd
import os

os.chdir(pybamm.__path__[0]+'/..')

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 561 kB 44.6 MB/s 
     |████████████████████████████████| 139 kB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 41 kB 189 kB/s 
     |████████████████████████████████| 34.2 MB 38 kB/s 
     |████████████████████████████████| 6.2 MB 40.6 MB/s 
     |████████████████████████████████| 4.3 MB 13.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


## Solve a model with a drive cycle

Load model

In [3]:
model = pybamm.lithium_ion.DFN()

Set up drive cycle

In [4]:
# import drive cycle from file
drive_cycle = pd.read_csv(
    "pybamm/input/drive_cycles/US06.csv", comment="#", header=None
).to_numpy()
# create interpolant
param = model.default_parameter_values
timescale = param.evaluate(model.timescale)
current_interpolant = pybamm.Interpolant(drive_cycle[:, 0], drive_cycle[:, 1], timescale * pybamm.t)
# set drive cycle
param["Current function [A]"] = current_interpolant

Create and run simulation using the CasadiSolver in "fast" mode, remembering to pass in the updated parameters

In [5]:
sim_US06_1 = pybamm.Simulation(
    model, parameter_values=param, solver=pybamm.CasadiSolver(mode="fast")
)
sol_US06_1 = sim_US06_1.solve()

## Update initial conditions based on a solution and solve again

Now pre-charge with CCCV, update the initial conditions, and solve again with the US06 drive cycle

In [6]:
experiment = pybamm.Experiment(
    ["Charge at 1 A until 4.1 V", "Hold at 4.1 V until 50 mA"]
)
sim_cccv = pybamm.Simulation(model, experiment=experiment)
sol_cccv = sim_cccv.solve()

# MODEL RE-INITIALIZATION: #############################################################
# Now initialize the model with the solution of the charge, and then discharge with
# the US06 drive cycle
# We could also do this inplace by setting inplace to True, which modifies the original
# model in place
new_model = model.set_initial_conditions_from(sol_cccv, inplace=False)
########################################################################################

sim_US06_2 = pybamm.Simulation(
    new_model, parameter_values=param, solver=pybamm.CasadiSolver(mode="fast")
)
sol_US06_2 = sim_US06_2.solve()

Plot both solutions, we can clearly see the difference now that initial conditions have been updated

In [7]:
pybamm.dynamic_plot(
    [sol_US06_1, sol_US06_2], labels=["Default initial conditions", "Fully charged"]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

## Initialize using a different model

We can also initialize the model using the solution of a different model

In [8]:
spm = pybamm.lithium_ion.SPM()
sim_spm_cccv = pybamm.Simulation(spm, experiment=experiment)
sol_spm_cccv = sim_spm_cccv.solve()

# MODEL RE-INITIALIZATION: #############################################################
# Now initialize the model with the solution of the charge, and then discharge with
# the US06 drive cycle
# We could also do this inplace by setting inplace to True, which modifies the original
# model in place
new_dfn = model.set_initial_conditions_from(sol_spm_cccv, inplace=False)
########################################################################################

sim_US06_3 = pybamm.Simulation(
    new_dfn, parameter_values=param, solver=pybamm.CasadiSolver(mode="fast")
)
sol_US06_3 = sim_US06_3.solve()

Now the model initialized by the DFN and the model initialized by the SPM give the same solution

In [9]:
pybamm.dynamic_plot(
    [sol_US06_1, sol_US06_2, sol_US06_3], 
    labels=["Default initial conditions", "Fully charged (from DFN)", "Fully charged (from SPM)"]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

## References

The relevant papers for this notebook are:

In [10]:
pybamm.print_citations()

PluginNotFound: ignored